In [1]:
### importing all the libraries needed
import time
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import requests
import csv

In [2]:
%%time

driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

sold_out = pd.read_csv('sold_out.csv')

indices = list(sold_out['index'])

original_price = []
categories = []
brand = []

for i in sold_out.index:
    try:
        if sold_out.loc[i,'URL'].startswith('https://shopee.sg/'):
            driver.get(sold_out.URL[i]) #get URL
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link-to-keep-parent-style"))).click()
        
            sleep(1)
        
            selenium_html = driver.page_source
            soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing

           
            cat_test = [x.get_text() for x in soup.find_all(class_ = "_3QRNmL")]
            cat_stack = ','.join(cat_test) 
            categories.append(cat_stack)

            brand_test = [x.get_text() for x in soup.find_all(class_ = "_3Qy6bH")]
            if len(brand_test)==0:
                brand.append('no brand')
            else:
                brand.append([x.get_text() for x in soup.find_all(class_ = "_3Qy6bH")][0])

            ori_price_test = [x.get_text() for x in soup.find_all(class_ = "_2MaBXe")]
            if len(ori_price_test)==0:
                original_price.append('no actual price')
            else:
                original_price.append([x.get_text() for x in soup.find_all(class_ = "_2MaBXe")][0])  
                
                df_sold_out = pd.DataFrame(list(zip(indices, original_price, brand, categories)),\
                                           columns =['Index', 'Original Price', 'Brand', 'Categories'])

    except:
        pass
    
driver.quit() #to automatically close the browser



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107]


Wall time: 32min 11s


In [3]:
df_sold_out

,Index,Original Price,Brand,Categories
0,0,$14.00,no brand,"Home & Living,Tools, DIY & Outdoors,Irrigation..."
1,1,$1.99,no brand,"Health & Wellness,OTC Medications,Eye, Nose, T..."
2,2,$4.30,no brand,"Jewellery & Accessories,Hairwear,Headbands,Uni..."
3,3,$2.01,no brand,"Jewellery & Accessories,Fashion Jewellery,Earr..."
4,4,no actual price,Huawei,"Computers & Peripherals,Monitors,29 Inches - 3..."
...,...,...,...,...
285,11816,$3.96,no brand,"Jewellery & Accessories,Hairwear,Hair Ties & S..."
286,11817,$1.80,no brand,"Kids Fashion,Kids Bags & Accessories,Hair Acce..."
287,11826,$3.00 - $3.40,no brand,"Home & Living,Lighting,Fairy Light,1M 2M Mini ..."
288,11827,$1.21,no brand,"Beauty & Personal Care,Beauty Devices & Tools,..."


In [4]:
df_sold_out.to_csv('sold_out_cat.csv')